In [1]:
#安裝套件
!pip install pyserini
!pip install sentence-transformers

  Using cached sentence_transformers-2.6.1-py3-none-any.whl (163 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64

In [2]:
import pandas as pd
from google.colab import drive

#檢查資料
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IR/train.txt', sep="\n#", header=None)
data

Mounted at /content/drive


<ipython-input-2-bcd4c3c7743b>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IR/train.txt', sep="\n#", header=None)


,0
0,###24491034
1,BACKGROUND\tThe emergence of HIV as a chronic ...
2,BACKGROUND\tThis paper describes the design an...
3,METHODS\tThis study is designed as a randomise...
4,METHODS\tThe intervention group will participa...
...,...
2402510,"METHODS\tAfter crossover , the wait-list group..."
2402511,METHODS\tPerformance was assessed by compariso...
2402512,RESULTS\tPerformance improved significantly af...
2402513,RESULTS\tNo improvement was detected as a func...


1. 使用 Python 命令執行 Pyserini 中的 index.lucene 模組
1. 指定要索引的文檔類型為 JSON 格式
1. 指定要索引的文檔目錄
1. 指定索引的輸出目錄和名稱
1. 指定文檔索引時使用的生成器。在此情況下，使用默認的 Lucene 文檔生成器
1. 指定用於索引的線程數。在此情況下，使用單線程
1. 指定存儲索引中的位置信息、文檔向量和原始文檔的選項


In [3]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input tests/resources \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

!pip install faiss-cpu
!apt-get purge openjdk-\* -qq > /dev/null
!apt-get autoremove -qq > /dev/null
!apt-get autoclean -qq > /dev/null
!apt-get install openjdk-21-jdk-headless -qq > /dev/nul
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.10/dist-packages/pyserini/index/__init__.py", line 21, in <module>
    from .lucene._base import Document, Generator, IndexTerm, Posting, IndexReader
  File "/usr/local/lib/python3.10/dist-packages/pyserini/index/lucene/__init__.py", line 17, in <module>
    from ._base import Document, Generator, IndexTerm, Posting, IndexReader
  File "/usr/local/lib/python3.10/dist-packages/pyserini/index/lucene/_base.py", line 30, in <module>
    from pyserini.analysis import get_lucene_analyzer, JAnalyzer, JAnalyzerUtils
  File "/usr/local/lib/python3.10/dist-packages/pyserini/analysis/__init__.py", line 17, in <module>
    from ._base import get_lucene_analyzer, Analyzer, JAnalyzer, JAnalyzerUtils, 

In [4]:
import json
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import time
import faiss
import numpy as np
from pyserini.search.lucene import LuceneSearcher

In [5]:
def load_documents(input_file_path):
    documents = []
    with open(input_file_path, 'r', encoding='utf-8') as input_file:
        document_id = None
        document_content = ''
        for line in input_file:
            if line.startswith('###'):
                if document_id is not None:
                    documents.append((document_id, document_content.strip()))
                document_id = line.strip().replace('#', '')
                document_content = ''
            else:
                document_content += line.strip() + ' '
        if document_id is not None:
            documents.append((document_id, document_content.strip()))
    return documents

def save_documents(documents, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for document_id, content in documents:
            json.dump({"id": document_id, "contents": content}, output_file)
            output_file.write('\n')

def encode_documents(model, documents, max_count=20000):
    embeddings = []
    for _, content in tqdm(documents[:max_count], desc="Encoding documents"):
        embedding = model.encode([content])[0]
        embeddings.append(embedding)
    return embeddings

def index_embeddings(embeddings):
    """建立FAISS索引。"""
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index

def search_embeddings(index, query_embedding, size):
    """使用索引進行檢索。"""
    D, I = index.search(np.array([query_embedding]), size)
    return D, I

def save_and_display_results(documents, index_results, query_word, output_file_path):
    """保存並顯示檢索結果"""
    D, I = index_results
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(f"包含詞彙 '{query_word}' 的前 {len(I[0])} 篇文檔：\n")
        for i, doc_idx in enumerate(I[0]):
            output_file.write(f"文檔索引：{doc_idx}, 相似度：{D[0][i]}\n")
            output_file.write(documents[doc_idx][1] + '\n')  # 保存文檔內容
            output_file.write("------------------------------\n")
            print(f"文檔索引：{doc_idx}, 相似度：{D[0][i]}")
            print(documents[doc_idx][1])
            print("------------------------------")



In [9]:
import torch
if __name__ == '__main__':
    input_file_path = '/content/drive/MyDrive/Colab Notebooks/IR/train.txt'
    output_file_path = '/content/drive/MyDrive/Colab Notebooks/IR/pubmed_20k.jsonl'

    model = SentenceTransformer('all-MiniLM-L6-v2')
    model = model.to(torch.device("cuda"))
    documents = load_documents(input_file_path)
    save_documents(documents, output_file_path)
    embeddings = encode_documents(model, documents)
    index = index_embeddings(embeddings)

    query_word = "AIDS"
    query_embedding = model.encode([query_word])[0]
    search_sizes = [5, 10]

    for size in search_sizes:
        D, I = search_embeddings(index, query_embedding, size)
        results_path = f'/content/drive/MyDrive/Colab Notebooks/IR/hw5_result_{size}.txt'
        save_and_display_results(documents, (D, I), query_word, results_path)

Encoding documents: 100%|██████████| 20000/20000 [03:27<00:00, 96.33it/s]


文檔索引：10209, 相似度：0.9502175450325012
OBJECTIVE	To test the effects of an HIV/AIDS education program . METHODS	A quasi-experimental , nested cross-sectional design including baseline and 6-month follow-up surveys . METHODS	Schools , stratified according to location , were randomly assigned to intervention ( n = 6 ) or comparison conditions ( n = 12 ) . METHODS	Public primary schools in Arusha and Kilimanjaro regions , Tanzania . METHODS	A total of 2026 sixth and seventh grade pupils ( average age , 14.0 years ) participated at baseline ( 85 % ) and 1785 at follow-up . METHODS	The program was designed to reduce children 's risk of HIV infection and to improve their tolerance of and care for people with AIDS . METHODS	Local teachers and health workers attended a 1-week training workshop before implementing the program over a 2-3-month period ( averaging 20 school hours per class ) . METHODS	Self-reported exposure to AIDS information , communication regarding AIDS ; AIDS knowledge , attitude